In [2]:
from utils.config_loader import load_config
from processing.preprocessing import preprocess_data
from processing.normalize import normalize_data
from processing.labelizer import get_labels
from model.train import train_model

config = load_config()

In [2]:
preprocess_data(config)

In [3]:
normalize_data(config)

In [4]:
get_labels(config)

In [3]:
train_model(config)

Epoch [1/2], Loss: 1.1549029350280762
Epoch [2/2], Loss: 0.7676583528518677
Epoch [3/2], Loss: 0.6442779302597046
